In [24]:
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


                


In [25]:
#train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()     # x에 rating 데이터를 복사한다. x = rating 대신에 copy 함수를 사용한 이유는 원본을 보존하기 위해서.
y = ratings['user_id'] # 기존의 데이터를 train set과 test set으로 분리할 떄 user_id 기준으로하는 stratified sampling 방식을 사용하기 떄문이다.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)



In [26]:
from sklearn.metrics import mean_squared_error

# 정확도 계산 

def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 모델별 RMSE를 계산하는 함수 

def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) #예측 대상인 test set에 있는 user_id와 movie_id를 pair로 짝을 맞춰 데이터를 만듬.
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs]) # 모든 user-movie 짝에 대해서 주어진 예측 모델에 의한 예측값을 계산해서 y_pred에 리스트로 저장
    y_true = np.array(x_test['rating'])        # 실제 평점값 리스트를 x_test에서 받아온다.
    return RMSE(y_true, y_pred)  #얻어진 예측값 리스트와 실제값 리스트에 대한 rmse를 계산해서 넘긴다.

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating') #pivot 메소드를 활용해서 train set의 full matrix를 구한다.



In [27]:
# 전체 평균으로 예측치를 계산하는 기본 모델

def best_seller(user_id, movie_id):   #해당 사용자의 해당 영화에 대한 평점 예측치 반환.
    try :                             #여기서 try - except를 사용하는 이유는 간혹 train set(x_train)에는 없는데 test set에는 존재하는 영화로 인해 발생하는 오류 방지위해서
        rating = train_mean[movie_id] #해당 영화가 train_mean(평균데이터)에 존재한다면 평균값을 돌려준다.
    except:
       
        rating = 3.0                  #해당 영화가 train_mean(평균데이터)에 존재하지 않는다면, 기본값 3.0을 돌려준다.
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean()   #train-seller 모델을 사용하기 위해서 영화의 전체에 대한 평균을 구한다.
score(best_seller) #기본 모델인 best-seller 방식으로 예측한 경우의 rmse를 계산한다. 여기서는 best_seller 함수가 모델이다.


1.024164324851864